# Imports

In [74]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.linear_model import LinearRegression

# Data

In [104]:
data = pd.read_csv('sphist.csv')

In [105]:
data['Date'] = pd.to_datetime(data['Date'])

In [106]:
data = data.sort_values('Date')

In [107]:
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


# Indicators

### 5 Day

In [108]:
# Average
data['5 day average'] = data['Close'].rolling(5).mean().shift(1)

In [109]:
# Standard deviation
data['5 day stdev'] = data['Close'].rolling(5).std().shift(1)

### 1 year

In [110]:
# Avereage
data['1 year average'] = data['Close'].rolling(365).mean().shift(1)

In [111]:
# Standard deviation
data['1 year stdev'] = data['Close'].rolling(365).std().shift(1)

### Ratios

In [112]:
def get_ratio(row, value):
    return row['5 day ' + value] / row['1 year ' + value]

In [113]:
data['Relative average'] = data.apply(lambda row: get_ratio(row, 'average'), axis=1)

In [114]:
data['Relative stdev'] = data.apply(lambda row: get_ratio(row, 'stdev'), axis=1)

In [115]:
cleaned_data = data.dropna()

In [116]:
cleaned_data.head()

,Date,Open,High,Low,Close,Volume,Adj Close,5 day average,5 day stdev,1 year average,1 year stdev,Relative average,Relative stdev
16224,1951-06-19,22.020000,22.020000,22.020000,22.020000,1100000.0,22.020000,21.800,0.256223,19.447726,1.790253,1.120954,0.143121
16223,1951-06-20,21.910000,21.910000,21.910000,21.910000,1120000.0,21.910000,21.900,0.213659,19.462411,1.789307,1.125246,0.119409
16222,1951-06-21,21.780001,21.780001,21.780001,21.780001,1100000.0,21.780001,21.972,0.092574,19.476274,1.788613,1.128142,0.051758
16221,1951-06-22,21.549999,21.549999,21.549999,21.549999,1340000.0,21.549999,21.960,0.115108,19.489562,1.787659,1.126757,0.064390
16220,1951-06-25,21.290001,21.290001,21.290001,21.290001,2440000.0,21.290001,21.862,0.204132,19.502082,1.786038,1.121008,0.114293


# Model

In [118]:
train = cleaned_data[cleaned_data['Date'] < datetime(year=2013, month=1, day=1)].drop(columns=['High', 
                                                                               'Low', 
                                                                               'Open', 
                                                                               'Volume', 
                                                                               'Adj Close', 
                                                                               'Date'])

In [119]:
test = cleaned_data[cleaned_data['Date'] >= datetime(year=2013, month=1, day=1)].drop(columns=['High', 
                                                                               'Low', 
                                                                               'Open', 
                                                                               'Volume', 
                                                                               'Adj Close', 
                                                                               'Date'])

In [120]:
lr = LinearRegression()

In [121]:
lr.fit(train.drop(columns=['Close']), train['Close'])


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [122]:
predictions = lr.predict(test.drop(columns=['Close']))

In [123]:
mse = abs(predictions - test['Close']).mean()
mse

16.131123821169925

# Add features

In [124]:
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close,5 day average,5 day stdev,1 year average,1 year stdev,Relative average,Relative stdev
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN,NaN,NaN,NaN
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN,NaN,NaN,NaN
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN,NaN,NaN,NaN
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN,NaN,NaN,NaN
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN,NaN,NaN,NaN,NaN,NaN


### Dummy months

In [140]:
data['Month'] = data.apply(lambda row: row['Date'].month, axis=1)

In [142]:
dummy_months = pd.get_dummies(data['Month'])

In [143]:
data = pd.concat([data, dummy_months], axis=1)

In [145]:
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close,5 day average,5 day stdev,1 year average,...,3,4,5,6,7,8,9,10,11,12
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


### Relative price to highest in year

In [146]:
data['Year max'] = data['Close'].rolling(365).max().shift(1)

In [148]:
data['Relative to year max'] = data.apply(lambda row: row['Year max'] / row['Close'], axis=1)

# Retrain

In [150]:
recleaned_data = data.dropna()

In [157]:
recleaned_data.columns

Index([                'Date',                 'Open',                 'High',
                        'Low',                'Close',               'Volume',
                  'Adj Close',        '5 day average',          '5 day stdev',
             '1 year average',         '1 year stdev',     'Relative average',
             'Relative stdev',                'Month',                      1,
                            2,                      3,                      4,
                            5,                      6,                      7,
                            8,                      9,                     10,
                           11,                     12,             'Year max',
       'Relative to year max'],
      dtype='object')

In [158]:
retrain = recleaned_data[recleaned_data['Date'] < datetime(year=2013, month=1, day=1)] \
.drop(columns=['High', 
               'Low', 
               'Open', 
               'Volume', 
               'Adj Close', 
               'Date',
              'Month',
              'Year max'])

In [159]:
retest = recleaned_data[recleaned_data['Date'] >= datetime(year=2013, month=1, day=1)] \
.drop(columns=['High', 
               'Low', 
               'Open', 
               'Volume', 
               'Adj Close', 
               'Date',
              'Month',
              'Year max'])

In [160]:
lr.fit(retrain.drop(columns=['Close']), retrain['Close'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [162]:
repredict = lr.predict(retest.drop(columns=['Close']))

In [163]:
mse = abs(repredict - retest['Close']).mean()
mse

15.629900881106574

Small improvements